<a href="https://colab.research.google.com/github/Kavya2099/recommendation-system/blob/main/Content_based_Movie_recommendation_from_kaggle_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Importing dataset from kaggle



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
! pip install -q kaggle
! pip install opendatasets
import opendatasets as od

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
od.download(
    "https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kavya2099
Your Kaggle Key: ··········


100%|██████████| 8.89M/8.89M [00:00<00:00, 120MB/s]

In [4]:
file = ('/content/tmdb-movie-metadata/tmdb_5000_movies.csv')
movie= pd.read_csv(file)

In [23]:
movie.shape

(4803, 20)

In [48]:
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",pirates of the caribbean: at world's end,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,the dark knight rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",john carter,6.1,2124


We are going to find the movie recommendation by using the overiew column of the dataset

So when we enter the movie title, model will provide the results based on the similarity with the overview column

Here we are doing **content-based filtering**




In [49]:
"""Using TfidfVectorizer

Transforms text to feature vectors that can be used as input to estimator"""


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# we are using stop_words to remove 'in', 'the' kind of words

vectorizer = TfidfVectorizer(stop_words="english",ngram_range=(1,2))


# here we are going to predict the movie recommendation using the overview column
movie.overview=movie.overview.fillna("")
tfidf = vectorizer.fit_transform(movie.overview)

Here we are creating the index of the list **indice** to be title of the movie. 

In [50]:
#changing the titles to lower case for better prediction

movie.title= movie.title.str.lower()

indice= pd.Series(movie.index, index=movie.title).drop_duplicates()
indice

title
avatar                                         0
pirates of the caribbean: at world's end       1
spectre                                        2
the dark knight rises                          3
john carter                                    4
                                            ... 
el mariachi                                 4798
newlyweds                                   4799
signed, sealed, delivered                   4800
shanghai calling                            4801
my date with drew                           4802
Length: 4803, dtype: int64

**Cosine similarity** is a metric used to measure the similarity of two vectors. Specifically, it measures the similarity in the direction or orientation of the vectors ignoring differences in their magnitude or scale.

Both vectors need to be part of the same inner product space, meaning they must produce a scalar through inner product multiplication. The similarity of two vectors is measured by the cosine of the angle between them.

In [51]:
""" the result of the cos_sim is if we provide the title index of 1, the
 cos_sim will be like ---> [1 0 0.12 ... ], which denotes that the first title is the best recommendation"""


from sklearn.metrics.pairwise import cosine_similarity

#creating a cosine similarity with the vectorizer 
cos_sim= cosine_similarity(tfidf,tfidf)



In [52]:
# creating function

def get_recommendation1(title,cos_sim=cos_sim):
    #changing title to lowercase
    
    title= title.lower()
    #passing the value of the title
    idx= indice[title] 

    #print(cos_sim[idx])
    #the reason for enumeration is to get the index of the sim_scores 
    sim_scores= enumerate(cos_sim[idx])

    #sorting the sim_scores based on the sim_scores in descending order
    sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)

    #since the first output will be the actual title, we will ignore that and start from the next title recommendation based on the cosine simularity
    sim_scores= sim_scores[1:11]
    
    #getting the index of the sim_scores
    sim_index= [i[0] for i in sim_scores]

    #printing the movies 
    print(movie['original_title'].iloc[sim_index])

In [53]:
#passing the title
get_recommendation1('Toy Story')

42                 Toy Story 3
343                Toy Story 2
1779    The 40 Year Old Virgin
3379              Factory Girl
3873             Class of 1984
891            Man on the Moon
2869    For Your Consideration
3065                Heartbeeps
3383                 Losin' It
2569               Match Point
Name: original_title, dtype: object


There is one disadvantage in the above mentioned model created. If we give the title which are not actually on the dataset, we'll get error on the function step ---> **idx= indice[title]**


The reason is that it cannot assign idx value since the title does not come under our dataset

So, for alternate solution, we can add if condition to be... like
if title is not found on the dataset/ title is misspelled, provide output to be --> 

**Given input title is either misspelled or not found in our dataset for movie recommendation. Kindly provide different title. Thank you**

In [54]:
from logging import exception
#Improved model

def get_recommendation(title,cos_sim=cos_sim):
  try:
    #changing title to lowercase
    title= title.lower()
    #passing the value of the title
    idx= indice[title] 

    #print(cos_sim[idx])
    #the reason for enumeration is to get the index of the sim_scores 
    sim_scores= enumerate(cos_sim[idx])

    #sorting the sim_scores based on the sim_scores in descending order
    sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)

    #since the first output will be the actual title, we will ignore that and start from the next title recommendation based on the cosine simularity
    sim_scores= sim_scores[1:11]
    
    #getting the index of the sim_scores
    sim_index= [i[0] for i in sim_scores]

    #printing the movies
    print(movie['original_title'].iloc[sim_index])
  except KeyError:
    print('Given input title is either misspelt or not found in our dataset for movie recommendation. Kindly provide different title. Thank you')

In [55]:
get_recommendation("Anabelle")

Given input title is either misspelt or not found in our dataset for movie recommendation. Kindly provide different title. Thank you


In [56]:
get_recommendation("Toy story")

42                 Toy Story 3
343                Toy Story 2
1779    The 40 Year Old Virgin
3379              Factory Girl
3873             Class of 1984
891            Man on the Moon
2869    For Your Consideration
3065                Heartbeeps
3383                 Losin' It
2569               Match Point
Name: original_title, dtype: object


In [60]:
get_recommendation("finding Nemo")

3124                                             The Reef
3568                         20,000 Leagues Under the Sea
525                                            Shark Tale
2069                     Teenage Mutant Ninja Turtles III
3389                                Chairman of the Board
1827    Teenage Mutant Ninja Turtles II: The Secret of...
3697                         Fast Times at Ridgemont High
2233                                           Little Boy
326                                            Cinderella
2742                                         The Shallows
Name: original_title, dtype: object
